# Basic Examples with Different Protocols

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
import json

## Seldon Protocol REST Model

In [ ]:
!pygmentize resources/model_seldon_rest.yaml

In [ ]:
!kubectl apply -f resources/model_seldon_rest.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=rest-seldon -o jsonpath='{.items[0].metadata.name}')

In [ ]:
for i in range(60):
    state=!kubectl get sdep rest-seldon -o jsonpath='{.status.state}'
    state=state[0]
    print(state)
    if state=="Available":
        break
    time.sleep(1)
assert(state=="Available")

In [ ]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/rest-seldon/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)
assert(d["data"]["ndarray"][0][0] > 0.4)

In [ ]:
!kubectl delete -f resources/model_seldon_rest.yaml

## Seldon Protocol GRPC Model

In [ ]:
!pygmentize resources/model_seldon_grpc.yaml

In [ ]:
!kubectl apply -f resources/model_seldon_grpc.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=grpc-seldon \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
for i in range(60):
    state=!kubectl get sdep grpc-seldon -o jsonpath='{.status.state}'
    state=state[0]
    print(state)
    if state=="Available":
        break
    time.sleep(1)
assert(state=="Available")

In [ ]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0]]}}' \
         -rpc-header seldon:grpc-seldon -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)
assert(d["data"]["ndarray"][0][0] > 0.4)

In [ ]:
!kubectl delete -f resources/model_seldon_grpc.yaml

## Tensorflow Protocol REST Model

In [ ]:
!pygmentize resources/model_tfserving_rest.yaml

In [ ]:
!kubectl apply -f resources/model_tfserving_rest.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=rest-tfserving \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
for i in range(60):
    state=!kubectl get sdep rest-tfserving -o jsonpath='{.status.state}'
    state=state[0]
    print(state)
    if state=="Available":
        break
    time.sleep(1)
assert(state=="Available")

In [ ]:
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/rest-tfserving/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

In [ ]:
!kubectl delete -f resources/model_tfserving_rest.yaml

## Tensorflow Protocol GRPC Model

In [ ]:
!pygmentize resources/model_tfserving_grpc.yaml

In [ ]:
!kubectl apply -f resources/model_tfserving_grpc.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=grpc-tfserving \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
for i in range(60):
    state=!kubectl get sdep grpc-tfserving -o jsonpath='{.status.state}'
    state=state[0]
    print(state)
    if state=="Available":
        break
    time.sleep(1)
assert(state=="Available")

In [ ]:
X=!cd ../executor/proto && grpcurl \
   -d '{"model_spec":{"name":"halfplustwo"},"inputs":{"x":{"dtype": 1, "tensor_shape": {"dim":[{"size": 3}]}, "floatVal" : [1.0, 2.0, 3.0]}}}' \
   -rpc-header seldon:grpc-tfserving -rpc-header namespace:seldon \
   -plaintext -proto ./prediction_service.proto \
   0.0.0.0:8003 tensorflow.serving.PredictionService/Predict
d=json.loads("".join(X))
print(d)
assert(d["outputs"]["x"]["floatVal"][0] == 2.5)

In [ ]:
!kubectl delete -f resources/model_tfserving_grpc.yaml